In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
import os

hf_token = os.environ.get('HF_TOKEN')
anthropic_api_key = os.environ.get('ANTHROPIC_API_KEY')

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Coder-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
rank = 256
alpha = rank
model = FastLanguageModel.get_peft_model(
    model,
    r = rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = alpha,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
alpaca_prompt = """### Instruction:
You are a code completion assistant and your task is to analyze user edits and then rewrite an excerpt that the user provides, suggesting the appropriate edits within the excerpt, taking into account the cursor location.

### User Edits:

{}

### User Excerpt:

{}

### Response:

{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
original_start_marker = "<|editable_region_start|>"
original_end_marker = "<|editable_region_end|>"

def format_example(events, input, output):
    return alpaca_prompt.format(events, input, output)

def formatting_prompts_func(examples):
    events       = examples["events"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for events, input, output in zip(events, inputs, outputs):
        output_start_index = output.find(original_start_marker)
        output_focused_region = output[output_start_index:]
        output = output_focused_region

        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = format_example(events, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

def filter_long_sequences(examples):
    tokenized = tokenizer(examples["text"])
    return len(tokenized['input_ids']) <= 1500

from datasets import load_dataset

# Pinned commit: 5920488, num_steps = 60, ratio = 2e-4, batch=8, rank=256, alpha=256
revision = "5920488"
dataset = load_dataset("zed-industries/zeta", token = hf_token, revision = revision)
dataset = dataset.map(formatting_prompts_func, batched = True,)
train_dataset = dataset["train"].filter(filter_long_sequences)
eval_dataset = dataset["eval"]

print("train len", len(train_dataset))
print("eval len", len(eval_dataset))

print(train_dataset[313]["text"])

In [ ]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

response_template = "### Response:\n\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
num_train_epochs = 2

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    data_collator = collator,
        args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        eval_steps = 5,
        eval_strategy = "steps",
        do_eval = True,
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
!pip install anthropic

In [ ]:
FastLanguageModel.for_inference(model)

import anthropic
from IPython.display import HTML
import json
from tqdm.notebook import tqdm
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

client = anthropic.Anthropic(api_key = anthropic_api_key)

def generate_model_output(events, input, model, tokenizer, alpaca_prompt):
    """Generate a single output using the model"""
    prompt = format_example(events, input, "")
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    prompt_length = inputs.input_ids.shape[1]
    outputs = model.generate(**inputs, max_new_tokens=2048, temperature=0, use_cache=True, do_sample=False)
    generated_text = tokenizer.decode(outputs[0][prompt_length:], skip_special_tokens=True)
    return generated_text

def get_score(model_output, assertions):
    prompt = f"""Your task is to help me grade code test output.

Here is the test output:

```
{model_output}
```

Now, help me score the test output using the following criteria:

{assertions}

Based on these criteria, give the test output a score between 0 and 5.

- 0 means: test output doesn't meet any criteria
- 5 means: test output meets all criteria
- If the test output does not contain any of the text I said it "must" include, grade it 0.
- First, perform a short, succinct analysis of how the test output meets the criteria.
- On the last line of your response, write the grade as a single number. Nothing else.
- **Always** end with a grade. The last line **must** be a number between 0 and 5."""

    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        temperature=0,
        system="You are a helpful assistant that grades test output based on given criteria.",
        messages=[{"role": "user", "content": prompt}]
    )

    # Extract the last line and convert to float
    last_line = response.content[-1].text.strip().split('\n')[-1]
    try:
        return float(last_line), response.content[-1].text
    except ValueError:
        return 0, response.content[-1].text

def process_dataset(dataset, model, tokenizer, alpaca_prompt):
    results = []

    for item in tqdm(dataset):
        # Generate model output
        model_output = generate_model_output(
            item['events'],
            item['input'],
            model,
            tokenizer,
            alpaca_prompt
        )

        # Get score from Claude
        score, full_response = get_score(model_output, item['assertions'])

        # Store results
        results.append({
            'events': item['events'],
            'input': item['input'],
            'model_output': model_output,
            'assertions': item['assertions'],
            'score': score,
            'full_response': full_response
        })

    # Convert results to DataFrame for display
    return pd.DataFrame(results)


results = process_dataset(eval_dataset, model, tokenizer, alpaca_prompt)


In [ ]:
def display_results(df, threshold=4):
    from IPython.display import display, Markdown

    for idx, row in df.iterrows():
        status = "❌ FAILED" if row['score'] < threshold else "✅ PASSED"
        status_color = "red" if row['score'] < threshold else "green"

        display(Markdown(f"""
### Test Case {idx + 1} - <span style="color: {status_color}">{status}</span> (Score: {row['score']})


#### Model Output:
```
{row['model_output']}
```

#### Assertions:
```
{row['assertions']}
```

#### Evaluation:
```
{row['full_response']}
```
---
"""))

    # Summary statistics
    total = len(df)
    passed = sum(df['score'] >= threshold)
    pass_rate = (passed / total) * 100

    display(Markdown(f"""
## Summary
- Total Tests: {total}
- Passed: {passed}
- Failed: {total - passed}
- Pass Rate: {pass_rate:.1f}%
"""))

display_results(results)

In [ ]:
model.push_to_hub_merged("zed-industries/zeta-sft", tokenizer, save_method = "merged_16bit", token = hf_token)